In [2]:
import sys
sys.path.append("..")  # Add the parent directory of 'examples' to the Python path


## Addresses

Create an `Address` object from a _bech32-encoded_ string:

In [3]:
from multiversx_sdk import Address

address = Address.new_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

print("Address (bech32-encoded)", address.to_bech32())
print("Public key (hex-encoded):", address.to_hex())
print("Public key (hex-encoded):", address.pubkey.hex())

Address (bech32-encoded) erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or from a _hex-encoded_ string - note that you have to provide the address prefix, also known as the **HRP** (_human-readable part_ of the address):

In [4]:
address = Address.new_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1", "erd")

... or from a raw public key:

In [5]:
pubkey = bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = Address(pubkey, "erd")

Alternatively, you can use an `AddressFactory` (initialized with a specific **HRP**) to create addresses:

In [6]:
from multiversx_sdk import AddressFactory

factory = AddressFactory("erd")

address = factory.create_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
address = factory.create_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = factory.create_from_public_key(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))

Addresses can be converted from one representation to another as follows:

In [7]:
print(address.to_bech32())
print(address.to_hex())

erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


Getting the shard of an address:

In [8]:
from multiversx_sdk import AddressComputer

address_computer = AddressComputer(number_of_shards=3)
print("Shard:", address_computer.get_shard_of_address(address))

Shard: 1


Checking whether an address is a smart contract:

In [9]:
address = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

print("Is contract:", address.is_smart_contract())

Is contract: True


## EGLD / ESDT transfers

Create an EGLD transfer:

In [10]:
from multiversx_sdk import Transaction, TransactionsConverter

transaction = Transaction(
    sender="erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th",
    receiver="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000
)

transaction_converter = TransactionsConverter()
print(transaction_converter.transaction_to_dictionary(transaction))

{'nonce': 77, 'value': '1000000000000000000', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 50000, 'data': '', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}


In case you are using a **guarded** account you should also populate the `guardian` and `guardian_signature` fields after creating the transaction.

We'll see later how to [sign](#signing-objects) and [broadcast](#broadcasting-transactions) a transaction.

Create an EGLD transfer, but this time with a payload (data):

In [11]:
transaction = Transaction(
    sender="erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th",
    receiver="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000,
    data=b"for the book"
)

print(transaction_converter.transaction_to_dictionary(transaction))

{'nonce': 77, 'value': '1000000000000000000', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 50000, 'data': 'Zm9yIHRoZSBib29r', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}


Alternatively, we can create an EGLD transfer using a **transaction factory** (as we will see below, transaction factories are more commonly used). But before that, we have to create a configuration object (for any factory that we might use):

In [12]:
from multiversx_sdk import TransactionsFactoryConfig

config = TransactionsFactoryConfig(chain_id="D")

The **transaction factory** is parametrized at instantiation, and the transaction is obtained by invoking the `create_transaction...` method:

In [13]:
from multiversx_sdk import TransferTransactionsFactory

transfer_factory = TransferTransactionsFactory(config=config)
alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")

# With "data" field
transaction = transfer_factory.create_transaction_for_native_token_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    data="for the book"
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '1000000000000000000', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 68000, 'data': 'Zm9yIHRoZSBib29r', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: for the book


Create a single ESDT transfer:

In [14]:
from multiversx_sdk import Token, TokenTransfer

token = Token("TEST-8b028f")
transfer = TokenTransfer(token, 10000)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 410000, 'data': 'RVNEVFRyYW5zZmVyQDU0NDU1MzU0MmQzODYyMzAzMjM4NjZAMjcxMA==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: ESDTTransfer@544553542d386230323866@2710


Create a single NFT transfer:

Keep in mind, since we are sending a NFT, we **should** set the amount to `1`.

In [15]:
token = Token(identifier="TEST-38f249", nonce=1)
transfer = TokenTransfer(token=token, amount=1)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1213500, 'data': 'RVNEVE5GVFRyYW5zZmVyQDU0NDU1MzU0MmQzMzM4NjYzMjM0MzlAMDFAMDFAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: ESDTNFTTransfer@544553542d333866323439@01@01@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a single SFT transfer (almost the same as above, the only difference being that for the transfer we set the desired amount, as an integer):

In [16]:
token = Token(identifier="SEMI-9efd0f", nonce=1)
transfer = TokenTransfer(token=token, amount=5)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1213500, 'data': 'RVNEVE5GVFRyYW5zZmVyQDUzNDU0ZDQ5MmQzOTY1NjY2NDMwNjZAMDFAMDVAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA==', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: ESDTNFTTransfer@53454d492d396566643066@01@05@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a multiple ESDT / NFT transfer:

In [17]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(transaction))
print("Transaction data:", transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 1484000, 'data': 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAw', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: MultiESDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000


### Decoding Transactions

For example, when sending multiple ESDT and NFT tokens, the receiver field of the transaction is the same as the sender field and also the value is set to `0` because all the information is encoded in the `data` field of the transaction.

For decoding the data field we have a so called `TransactionDecoder`. We fetch the transaction from the network and then use the decoder.

In [18]:
from multiversx_sdk import ProxyNetworkProvider, TransactionDecoder

proxy = ProxyNetworkProvider("https://devnet-api.multiversx.com")
transaction = proxy.get_transaction("3e7b39f33f37716186b6ffa8761d066f2139bff65a1075864f612ca05c05c05d")

decoder = TransactionDecoder()
decoded_transaction = decoder.get_transaction_metadata(transaction)

print(decoded_transaction.to_dict())

{'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qqqqqqqqqqqqqpgqtqfhy99su9xzjjrq59kpzpp25udtc9eq0n4sr90ax6', 'value': 45465672416735998059, 'function_name': 'swapTokensFixedOutput', 'function_args': ['555344432d333530633465', '3b9aca00'], 'transfers': [{'value': 45465672416735998059, 'token': 'WEGLD-a28c59', 'nonce': 0}]}


## Relayed Transactions

First, we get the newtwork configuration using the network providers.

In [19]:
from multiversx_sdk import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")
network_config = provider.get_network_config()

### Relayed V1

In [20]:
from pathlib import Path

from multiversx_sdk import (Address, Transaction, TransactionComputer,
                            RelayedTransactionsFactory, TransactionsFactoryConfig,
                            UserSigner)

signer = UserSigner.from_pem_file(Path("../multiversx_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

inner_tx = Transaction(
    chain_id=network_config.chain_id,
    sender="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    receiver="erd1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssm4y34z",
    gas_limit=60000000,
    nonce=198,
    data=b"add@05"
)
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

relayed_tx = factory.create_relayed_v1_transaction(
    inner_transaction=inner_tx,
    relayer_address=relayer
)
relayed_tx.nonce = 2627

print(transaction_converter.transaction_to_dictionary(relayed_tx))

{'nonce': 2627, 'value': '0', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 61052000, 'data': 'cmVsYXllZFR4QDdiMjI2ZTZmNmU2MzY1MjIzYTMxMzkzODJjMjI3MzY1NmU2NDY1NzIyMjNhMjI2NzQ1NmU1NzRmNjU1NzZkNmQ0MTMwNjMzMDZhNmI3MTc2NGQzNTQyNDE3MDdhNjE2NDRiNDY1NzRlNTM0ZjY5NDE3NjQzNTc1MTYzNzc2ZDQ3NTA2NzNkMjIyYzIyNzI2NTYzNjU2OTc2NjU3MjIyM2EyMjQxNDE0MTQxNDE0MTQxNDE0MTQxNDE0NjQxNDE1OTQ2NGUyYjRkNGU1NTY5MzUzODY3NjI0MjMzMzUyYjYzNTY2NzczNGM1OTM5NzI2NzY1NjE2NTQ1M2QyMjJjMjI3NjYxNmM3NTY1MjIzYTMwMmMyMjY3NjE3MzUwNzI2OTYzNjUyMjNhMzEzMDMwMzAzMDMwMzAzMDMwMzAyYzIyNjc2MTczNGM2OTZkNjk3NDIyM2EzNjMwMzAzMDMwMzAzMDMwMmMyMjY0NjE3NDYxMjIzYTIyNTk1NzUyNmI1MTQ0NDEzMTIyMmMyMjczNjk2NzZlNjE3NDc1NzI2NTIyM2EyMjQ3NGM0MzM0NGE1OTM1NGMzMDc1NmY0YTRkNzA0ODc1MzQ0ODUwNTc2NzQ3NDE0ODQ3NmY0NzZhN2E0NzYxMzYzNTJiNzU0ODMwMzAyZjc3NGM1MTU5NDU1MTUyMzA2NDVhNGM2MzUxMzQ

### Relayed V2

In [21]:
from pathlib import Path

from multiversx_sdk import (Address, Transaction, TransactionComputer,
                            RelayedTransactionsFactory, TransactionsFactoryConfig,
                            UserSigner)

signer = UserSigner.from_pem_file(Path("../multiversx_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

# for the relayedV2 transactions, the gasLimit for the inner transaction should be 0
inner_tx = Transaction(
            chain_id=network_config.chain_id,
            sender="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
            receiver="erd1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssm4y34z",
            gas_limit=0,
            nonce=15,
            data=b"add@05"
        )
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

relayed_tx = factory.create_relayed_v2_transaction(
    inner_transaction=inner_tx,
    inner_transaction_gas_limit=60_000_000,
    relayer_address=relayer
)
relayed_tx.nonce = 37

print(transaction_converter.transaction_to_dictionary(relayed_tx))

The `serialize()` method is deprecated and will soon be removed


{'nonce': 37, 'value': '0', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 60381500, 'data': 'cmVsYXllZFR4VjJAMDAwMDAwMDAwMDAwMDAwMDA1MDAwNjA1MzdlMzBkNTIyZTdjODFiMDc3ZTdlNzE1ODJjMmQ4ZjZiODFlNjllMUAwZkA2MTY0NjQ0MDMwMzVANjUwNzJhOTExOGRiMTQ3NDdhNGYxY2Y4NTdhNmU0MzczZTNlOWVhZDZlYTYwMzZmNTExMWJmNjg4MDNjNWNhZGZkYzQ0ZDM3YTIxYjkyMDQ0OWI1ZTgzYzc4Mzk3NTQyNWMzZjQ3NjAyMDE3M2FjMGZjNzE0ODE2NDZkNGE5MDA=', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}


## Contract deployments and interactions

Create a transaction to deploy a smart contract:

In [22]:
from pathlib import Path

from multiversx_sdk import SmartContractTransactionsFactory

sc_factory = SmartContractTransactionsFactory(config)
bytecode = Path("./data/counter.wasm").read_bytes()

deploy_transaction = sc_factory.create_transaction_for_deploy(
    sender=alice,
    bytecode=bytecode,
    arguments=[42, "test"],
    gas_limit=10000000,
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", transaction_converter.transaction_to_dictionary(deploy_transaction))
print("Transaction data:", deploy_transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'MDA2MTczNmQwMTAwMDAwMDAxMTUwNDYwMDM3ZjdmN2UwMTdmNjAwMjdmN2YwMTdlNjAwMTdlMDA2MDAwMDAwMjQyMDMwMzY1NmU3NjExNjk2ZTc0MzYzNDczNzQ2ZjcyNjE2NzY1NTM3NDZmNzI2NTAwMDAwMzY1NmU3NjEwNjk2ZTc0MzYzNDczNzQ2ZjcyNjE2NzY1NGM2ZjYxNjQwMDAxMDM2NTZlNzYwYjY5NmU3NDM2MzQ2NjY5NmU2OTczNjgwMDAyMDMwNTA0MDMwMzAzMDMwNDA1MDE3MDAxMDEwMTA1MDMwMTAwMDIwNjA4MDE3ZjAxNDE5MDg4MDQwYjA3MmYwNTA2NmQ2NTZkNmY3Mjc5MDIwMDA0Njk2ZTY5NzQwMDAzMDk2OTZlNjM3MjY1NmQ2NTZlNzQwMDA0MDk2NDY1NjM3MjY1NmQ2NTZlNzQwMDA1MDM2NzY1NzQwMDA2MGE4YTAxMDQxMzAwNDE4MDg4ODA4MDAwNDEwNzQyMDExMDgwODA4MDgwMDAxYTBiMmUwMTAxN2U0MTgwODg4MDgwMDA0MTA3NDE4MDg4ODA4MDAwNDEwNzEwODE4MDgwODAwMDQyMDE3YzIyMDAxMDgwODA4MDgwMDAxYTIwMDAxMDgyODA4MDgwMDAwYjJlMDEwMTdlNDE4MDg4ODA4MDAwNDEwNzQxODA4ODgwO

Create a transaction to upgrade an existing smart contract:

In [23]:
contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")
bytecode = Path("./data/counter.wasm").read_bytes()

upgrade_transaction = sc_factory.create_transaction_for_upgrade(
    sender=alice,
    contract=contract_address,
    bytecode=bytecode,
    gas_limit=10000000,
    arguments=[42, "test"],
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", transaction_converter.transaction_to_dictionary(upgrade_transaction))
print("Transaction data:", upgrade_transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'dXBncmFkZUNvbnRyYWN0QDAwNjE3MzZkMDEwMDAwMDAwMTE1MDQ2MDAzN2Y3ZjdlMDE3ZjYwMDI3ZjdmMDE3ZTYwMDE3ZTAwNjAwMDAwMDI0MjAzMDM2NTZlNzYxMTY5NmU3NDM2MzQ3Mzc0NmY3MjYxNjc2NTUzNzQ2ZjcyNjUwMDAwMDM2NTZlNzYxMDY5NmU3NDM2MzQ3Mzc0NmY3MjYxNjc2NTRjNmY2MTY0MDAwMTAzNjU2ZTc2MGI2OTZlNzQzNjM0NjY2OTZlNjk3MzY4MDAwMjAzMDUwNDAzMDMwMzAzMDQwNTAxNzAwMTAxMDEwNTAzMDEwMDAyMDYwODAxN2YwMTQxOTA4ODA0MGIwNzJmMDUwNjZkNjU2ZDZmNzI3OTAyMDAwNDY5NmU2OTc0MDAwMzA5Njk2ZTYzNzI2NTZkNjU2ZTc0MDAwNDA5NjQ2NTYzNzI2NTZkNjU2ZTc0MDAwNTAzNjc2NTc0MDAwNjBhOGEwMTA0MTMwMDQxODA4ODgwODAwMDQxMDc0MjAxMTA4MDgwODA4MDAwMWEwYjJlMDEwMTdlNDE4MDg4ODA4MDAwNDEwNzQxODA4ODgwODAwMDQxMDcxMDgxODA4MDgwMDA0MjAxN2MyMjAwMTA4MDgwODA4MDAwMWEyMDAwMTA4MjgwODA4MDAwMGIyZTAxMDE3ZTQxODA4ODgwO

Create a transaction that invokes a smart contract function:

In [24]:
contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

call_transaction = sc_factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="foo",
    gas_limit=10000000,
    arguments=[42, "test"]
)

print("Transaction:", transaction_converter.transaction_to_dictionary(call_transaction))
print("Transaction data:", call_transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'Zm9vQDJhQDc0NjU3Mzc0', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: foo@2a@74657374


Now, let's create a call that also transfers one or more tokens (**transfer & execute**):

In [25]:
first_token = Token("TEST-38f249", 1)
first_transfer = TokenTransfer(first_token, 1)

second_token = Token("BAR-c80d29")
second_transfer = TokenTransfer(second_token, 10000000000000000000)

transfers = [first_transfer, second_transfer]

call_transaction = sc_factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="hello",
    gas_limit=10000000,
    arguments=[42, "test"],
    token_transfers=transfers
)

print("Transaction:", transaction_converter.transaction_to_dictionary(call_transaction))
print("Transaction data:", call_transaction.data.decode())

Transaction: {'nonce': 0, 'value': '0', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'senderUsername': '', 'receiverUsername': '', 'gasPrice': 1000000000, 'gasLimit': 10000000, 'data': 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAMDAwMDAwMDAwMDAwMDAwMDA1MDBlMGI3N2YxZWRmYjAxY2I3ODZhMzkxMjBmMDJjMzFmZjVmZTQwYWFkODk3NEAwMkA1NDQ1NTM1NDJkMzMzODY2MzIzNDM5QDAxQDAxQDQyNDE1MjJkNjMzODMwNjQzMjM5QEA4YWM3MjMwNDg5ZTgwMDAwQDY4NjU2YzZjNmZAMmFANzQ2NTczNzQ=', 'chainID': 'D', 'version': 2, 'options': 0, 'guardian': '', 'signature': '', 'guardianSignature': ''}
Transaction data: MultiESDTNFTTransfer@00000000000000000500e0b77f1edfb01cb786a39120f02c31ff5fe40aad8974@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@68656c6c6f@2a@74657374


## Contract queries

In order to do a contract query against the network (more details about **network providers** can be found below), do as follows:

In [26]:
from multiversx_sdk import (ProxyNetworkProvider, QueryRunnerAdapter,
                            SmartContractQueriesController)

contract = Address.from_bech32("erd1qqqqqqqqqqqqqpgqqy34h7he2ya6qcagqre7ur7cc65vt0mxrc8qnudkr4")
query_runner = QueryRunnerAdapter(ProxyNetworkProvider("https://devnet-api.multiversx.com"))

query_controller = SmartContractQueriesController(query_runner)

data_parts = query_controller.query(
    contract=contract.to_bech32(),
    function="getSum",
    arguments=[],
)

print("Return data (parsed):", data_parts)

Return data (parsed): [b'\x05']


For finer control, first create a contract query, then run it and parse the outcome at a later time:

In [56]:
query = query_controller.create_query(
    contract=contract.to_bech32(),
    function="getSum",
    arguments=[],
)

response = query_controller.run_query(query)
data_parts = query_controller.parse_query_response(response)

print("Return code:", response.return_code)
print("Return data (raw):", response.return_data_parts)
print("Return data (parsed):", data_parts)

Return code: ok
Return data (raw): [b'\x05']
Return data (parsed): [b'\x05']


## Creating wallets

Mnemonic generation is based on [`trezor/python-mnemonic`](https://github.com/trezor/python-mnemonic) and can be achieved as follows:

In [4]:
from multiversx_sdk import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

['duck', 'road', 'best', 'card', 'rapid', 'belt', 'shuffle', 'umbrella', 'trophy', 'donor', 'discover', 'maple', 'inherit', 'play', 'chat', 'need', 'siege', 'tiny', 'broom', 'ugly', 'margin', 'life', 'shove', 'frown']


The mnemonic can be saved to a keystore file:

In [8]:
from multiversx_sdk import UserWallet
from pathlib import Path

path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

wallet = UserWallet.from_mnemonic(mnemonic.get_text(), "password")
wallet.save(path / "walletWithMnemonic.json")

Given a mnemonic, one can derive keypairs:

In [10]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key:", secret_key.hex())
print("Public key:", public_key.hex())

Secret key: 2f0c8a8f516f820c09d1c13a35cf90eaedcb59a66203810b515c4c6162e0a234
Public key: b80a407e1a7e05ef0b5b91351b1818e695a2defb41ff211311f99aad9846394f


A keypair can be saved as a JSON wallet:

In [11]:
path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

wallet = UserWallet.from_secret_key(secret_key, "password")
wallet.save(path / "wallet.json", address_hrp="erd")

... or as a PEM wallet (usually not recommended):

In [13]:
from multiversx_sdk import Address, UserPEM

path = Path("./output")
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

label = Address(public_key.buffer, "erd").to_bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(path / "wallet.pem")

## Loading wallets

This is not a very common use-case - you might refer to [signing objects](#signing-objects) instead.

Load a keystore that holds an **encrypted mnemonic** (and perform wallet derivation at the same time):

In [31]:
from multiversx_sdk import UserWallet

secret_key = UserWallet.load_secret_key(Path("../multiversx_sdk/testutils/testwallets/withDummyMnemonic.json"), "password", address_index=0)
address = secret_key.generate_public_key().to_address("erd")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address: erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th


Load a keystore that holds an **encrypted secret** key:

In [32]:
secret_key = UserWallet.load_secret_key(Path("../multiversx_sdk/testutils/testwallets/alice.json"), "password")
address = secret_key.generate_public_key().to_address("erd")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address: erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th


Load the secret key from a PEM file:

In [33]:
from multiversx_sdk import UserPEM

pem = UserPEM.from_file(Path("../multiversx_sdk/testutils/testwallets/alice.pem"))

print("Secret key:", pem.secret_key.hex())
print("Public key:", pem.public_key.hex())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Public key: 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


## Signing objects

Creating a `UserSigner` from a JSON wallet:

In [34]:
from multiversx_sdk import UserSigner

signer = UserSigner.from_wallet(Path("../multiversx_sdk/testutils/testwallets/alice.json"), "password")

Creating a `UserSigner` from a PEM file:

In [35]:
signer = UserSigner.from_pem_file(Path("../multiversx_sdk/testutils/testwallets/alice.pem"))

Signing a transaction:

In [36]:
from multiversx_sdk import Transaction, TransactionComputer

tx = Transaction(
    nonce=90,
    sender="erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th",
    receiver="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    value=1000000000000000000,
    gas_limit=50000,
    chain_id="D"
)

transaction_computer = TransactionComputer()

tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))
print("Signature:", tx.signature.hex())

Signature: ee6d665981c00258858923bbbe51e1cfe29911716190a3eedc24e2205100bb05faf3d8133fc973b00409aa18708b41488702d635efefe6a24b4a2b7f66267a00


Signing an arbitrary message:

In [37]:
from multiversx_sdk import Message, MessageComputer

signer_address = signer.get_pubkey().to_address(hrp="erd")
message = Message(b"hello")
message_computer = MessageComputer()

message.signature = signer.sign(message_computer.compute_bytes_for_signing(message))

print("Signature:", message.signature.hex())

Signature: 561bc58f1dc6b10de208b2d2c22c9a474ea5e8cabb59c3d3ce06bbda21cc46454aa71a85d5a60442bd7784effa2e062fcb8fb421c521f898abf7f5ec165e5d0f


## Verifying signatures

Creating a `UserVerifier`:

In [38]:
from multiversx_sdk import Address, UserVerifier

alice = Address.new_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.new_from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")

alice_verifier = UserVerifier.from_address(alice)
bob_verifier = UserVerifier.from_address(bob)

Verifying a signature:

In [39]:
from multiversx_sdk import MessageComputer, TransactionComputer

transaction_computer = TransactionComputer()
message_computer = MessageComputer()

print(f"Is signature of Alice?", alice_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Alice?", alice_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))
print(f"Is signature of Bob?", bob_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Bob?", bob_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))

Is signature of Alice? True
Is signature of Alice? True
Is signature of Bob? False
Is signature of Bob? False


## Creating network providers

It's recommended to use the `multiversx_sdk_network_providers` components **as a starting point**. As your application matures, switch to using your own network provider (e.g. deriving from the default ones), tailored to your requirements.

Creating an API provider:

In [40]:
from multiversx_sdk import ApiNetworkProvider

provider = ApiNetworkProvider("https://devnet-api.multiversx.com")

Creating a Proxy provider:

In [41]:
from multiversx_sdk import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")

## Fetching network parameters

In order to fetch network parameters, do as follows:

In [42]:
config = provider.get_network_config()

print("Chain ID:", config.chain_id)
print("Min gas price:", config.min_gas_price)

Chain ID: D
Min gas price: 1000000000


## Fetching account state

The following snippet fetches (from the Network) the **nonce** and the **balance** of an account:

In [43]:
account_on_network = provider.get_account(alice)

print("Nonce:", account_on_network.nonce)
print("Balance:", account_on_network.balance)

Nonce: 1133
Balance: 17869817183859999837


When sending a number of transactions, you usually have to first fetch the account nonce from the network (see above), then manage it locally (e.g. increment upon signing & broadcasting a transaction):

In [44]:
from multiversx_sdk import AccountNonceHolder

nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx.nonce = nonce_holder.get_nonce_then_increment()
# Then, sign & broadcast the transaction(s).

For further reference, please see [nonce management](https://docs.multiversx.com/integrators/creating-transactions/#nonce-management).

## Broadcasting transactions

Broadcast a single transaction:

In [45]:
alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

tx = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)

tx.nonce = alice_on_network.nonce
tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))

hash = provider.send_transaction(tx)
print("Transaction hash:", hash)

Transaction hash: 4c57f147644bf21ad83c5e732e95dd6cebf56bded40827929130dfcc1bf721c5


Broadcast multiple transactions:

In [46]:
tx_1 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_2 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_3 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)
nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx_1.nonce = nonce_holder.get_nonce_then_increment()
tx_2.nonce = nonce_holder.get_nonce_then_increment()
tx_3.nonce = nonce_holder.get_nonce_then_increment()

tx_1.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_1))
tx_2.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_2))
tx_3.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_3))

hashes = provider.send_transactions([tx_1, tx_2, tx_3])
print("Transactions hashes:", hashes)

Transactions hashes: (3, {'0': '4c57f147644bf21ad83c5e732e95dd6cebf56bded40827929130dfcc1bf721c5', '1': '1dc19cc442afdc80bfe341120778b197bd24e5f4ff41511f9be8b882132060d8', '2': '795e81c21c4c613479ebdbfc466140d8c24574f71d672bcffd2e5fee0580da42'})


Now let's fetch a previously-broadcasted transaction:

In [47]:
tx_on_network = provider.get_transaction("9270a6879b682a7b310c659f58b641ccdd5f083e5633669817130269e5b0939b", with_process_status=True)
print("Status:", tx_on_network.status)
print("Is completed:", tx_on_network.is_completed)

Status: success
Is completed: True
